# Dataset loaded

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("/home/catpc/Downloads/archive (11)/IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [2]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [3]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [4]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [5]:
df.duplicated().sum()

418

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
df.duplicated().sum()

0

In [8]:
#Basic Preprocessing
#Remove tags
#lowercase
#remove stopwords


# Text Preprocessing

# HTML tag remove

In [9]:
import re 
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'),'',raw_text)
    return cleaned_text

In [10]:
df['review'] = df['review'].apply(remove_tags)

In [11]:
df['review'].head()

0    One of the other reviewers has mentioned that ...
1    A wonderful little production. The filming tec...
2    I thought this was a wonderful way to spend ti...
3    Basically there's a family where a little boy ...
4    Petter Mattei's "Love in the Time of Money" is...
Name: review, dtype: object

# Lowercase convert 

In [12]:
df['review'] = df['review'].apply(lambda x: x.lower())
df['review'].head()

0    one of the other reviewers has mentioned that ...
1    a wonderful little production. the filming tec...
2    i thought this was a wonderful way to spend ti...
3    basically there's a family where a little boy ...
4    petter mattei's "love in the time of money" is...
Name: review, dtype: object

# Stopwords remove

In [13]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [14]:
df

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. filming technique...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive
...,...,...
49995,thought movie right good job. creative origina...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,catholic taught parochial elementary schools n...,negative
49998,going disagree previous comment side maltin on...,negative


In [15]:
x = df.iloc[:,0:1]
x

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
49995,thought movie right good job. creative origina...
49996,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,catholic taught parochial elementary schools n...
49998,going disagree previous comment side maltin on...


In [16]:
y = df['sentiment']

In [17]:
y

0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
49995    positive
49996    negative
49997    negative
49998    negative
49999    negative
Name: sentiment, Length: 49582, dtype: object

In [18]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [19]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test , y_train, y_test = train_test_split(x,y , test_size=0.2 , random_state=1)

# Apply bow 

In [21]:
from sklearn.feature_extraction.text import  CountVectorizer
cv = CountVectorizer(max_features=1000)



In [22]:
x_train_bow = cv.fit_transform(x_train['review']).toarray()
x_test_bow = cv.transform(x_test['review']).toarray()

In [23]:
x_train_bow.shape

(39665, 1000)

In [24]:
from sklearn.naive_bayes import GaussianNB

# Step 1: Create the model
gnb = GaussianNB()

# Step 2: Train the model
gnb.fit(x_train_bow, y_train)

# Step 3: Predict on test data
y_pred = gnb.predict(x_test_bow)


In [25]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.813149137844106

In [26]:
confusion_matrix(y_test,y_pred)

array([[4087,  946],
       [ 907, 3977]])

In [27]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

mnb.fit(x_train_bow, y_train)

y_pred_mnb = mnb.predict(x_test_bow)


In [28]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred_mnb)

0.829686397095896

In [29]:
confusion_matrix(y_test,y_pred_mnb)

array([[4112,  921],
       [ 768, 4116]])

# model is  good or bad

# Accuracy

In [30]:
accuracy = (3977+4087)/(4087+946+907+4087)
accuracy # not bad

0.8042285828263688

# precision for class 1

In [31]:
precision = 3977/(3977+946)
precision

0.8078407475116799

# Recall for class 1

In [32]:
recall = 3977/(3977+907)
recall

0.8142915642915642

# imporvement of model

In [33]:
#incress max_feature
cv = CountVectorizer(max_features=3000)

x_train_bow = cv.fit_transform(x_train['review']).toarray()
x_test_bow = cv.transform(x_test['review']).toarray()



from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(x_train_bow,y_train)

y_pred_rf = rf.predict(x_test_bow)

accuracy_score(y_test,y_pred_rf)


0.8427952001613391

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(1, 2), max_features=3000)
x_train_bow = cv.fit_transform(x_train['review']).toarray()
x_test_bow = cv.transform(x_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(x_train_bow,y_train)

y_pred_rf = rf.predict(x_test_bow)

accuracy_score(y_test, y_pred_rf)

0.8448119390944843

# using TF-IDF

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features = 1000)


In [40]:
x_train_tfidf = tfidf.fit_transform(x_train['review']).toarray()
x_test_tfidf = tfidf.transform(x_test['review']).toarray()

rf = RandomForestClassifier()
rf.fit(x_train_tfidf,y_train)



RandomForestClassifier()

In [41]:
y_pred_tfidf = rf.predict(x_test_tfidf)
accuracy_score(y_test,y_pred_tfidf)

0.8328123424422709